## 1851. Minimum Interval to Include Each Query
- Description:
  <blockquote>
    You are given a 2D integer array `intervals`, where `intervals[i] = [lefti, righti]` describes the `ith` interval starting at `lefti` and ending at `righti` **(inclusive)**. The **size** of an interval is defined as the number of integers it contains, or more formally `righti- lefti+ 1`.
     
    You are also given an integer array `queries`. The answer to the `jth` query is the **size of the smallest interval** `i` such that `lefti<= queries[j] <= righti`. If no such interval exists, the answer is `-1`.
     
    Return  *an array containing the answers to the queries* .
     
    **Example 1:**
    **Input:** intervals = `[1, 4],[2,4],[3,6],[4,4]`, queries = [2,3,4,5]
    **Output:** [3,3,1,4]
    **Explanation:** The queries are processed as follows:
    - Query = 2: The interval [2,4] is the smallest interval containing 2. The answer is 4 - 2 + 1 = 3.
    - Query = 3: The interval [2,4] is the smallest interval containing 3. The answer is 4 - 2 + 1 = 3.
    - Query = 4: The interval [4,4] is the smallest interval containing 4. The answer is 4 - 4 + 1 = 1.
    - Query = 5: The interval [3,6] is the smallest interval containing 5. The answer is 6 - 3 + 1 = 4.
     
    **Example 2:**
    **Input:** intervals = `[2, 3],[2,5],[1,8],[20,25]`, queries = [2,19,5,22]
    **Output:** [2,-1,4,6]
    **Explanation:** The queries are processed as follows:
    - Query = 2: The interval [2,3] is the smallest interval containing 2. The answer is 3 - 2 + 1 = 2.
    - Query = 19: None of the intervals contain 19. The answer is -1.
    - Query = 5: The interval [2,5] is the smallest interval containing 5. The answer is 5 - 2 + 1 = 4.
    - Query = 22: The interval [20,25] is the smallest interval containing 22. The answer is 25 - 20 + 1 = 6.
     
    **Constraints:**
     
    - `1 <= intervals.length <= 105`
    - `1 <= queries.length <= 105`
    - `intervals[i].length == 2`
    - `1 <= lefti<= righti<= 107`
    - `1 <= queries[j] <= 107`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/minimum-interval-to-include-each-query/description/)

- Topics: Interval, Sorting, Heap

- Difficulty: Hard

- Resources: example_resource_URL

### Solution 1, Offline Queries with Min-Heap

"offline queries" means:
We are allowed to process all queries in any order we choose (typically sorted), rather than answering them in the original given order one by one as they arrive.

Online queries: You must answer each query immediately as it comes, without knowing future queries.
Offline queries: You can see all queries in advance, so you can reorder them (e.g., sort by value) to make processing more efficient.

If we sort queries by value (e.g., from smallest to largest), we can sweep through them while maintaining a dynamic set of "active" intervals using a min-heap.

This avoids checking all intervals for every query — leading to O((n + m) log n) instead of O(n·m).

For N intervals and M queries:
- Time Complexity: O(nlogn + mlogm)
  - Sorting intervals: O(n log n)
  - Sorting queries: O(m log m)
  - Each interval is pushed/popped from the heap once: O(n log n)
- Space Complexity: O(n+m)
  - qidxs: stores m indexes -> O(m)
  - result array stores m results -> O(m)
  - Min-heap: in worst case, holds all n intervals -> O(n)

In [ ]:
class Solution:
    def minInterval(self, intervals: List[List[int]], queries: List[int]) -> List[int]:
        # Sort intervals by left endpoint
        intervals.sort()
        
        # Create sorted list of queries with original indices
        # sorted_queries = sorted((q, i) for i, q in enumerate(queries))
        
        # Sorted query indices
        qidxs = list(range(len(queries)))
        qidxs.sort(key=lambda i: queries[i])
        
        # Min-heap to store (size, right)
        # Use a min-heap to track active intervals (those that have started and not yet ended). 
        min_heap = []
        
        # Result array
        result = [-1] * len(queries)
        
        # We never revisit earlier intervals because they’ve already been considered for smaller queries 
        # and remain in the heap if still valid (i.e., their right >= current query).
        interval_ptr = 0
        
        for idx in qidxs:
            query = queries[idx]
            # Add all intervals that start <= query
            while interval_ptr < len(intervals) and intervals[interval_ptr][0] <= query:
                l, r = intervals[interval_ptr]
                size = r - l + 1
                heapq.heappush(min_heap, (size, r))
                interval_ptr += 1
            
            # Remove intervals that end before q (no longer valid)
            # any interval whos right >= current query remain in the queue to be processed for the next query
            while min_heap and min_heap[0][1] < query:
                heapq.heappop(min_heap)
            
            # If heap is not empty, top element is the answer
            if min_heap:
                result[idx] = min_heap[0][0]
        
        return result

Alt sol similar to above, slightly faster

In [ ]:
class Solution:
    def minInterval(self, intervals: List[List[int]], queries: List[int]) -> List[int]:
        ans = [-1]*len(queries)

        # Sort intervals by start point
        intervals.sort()
        qidxs = list(range(len(queries)))
        # Sort the index of the queries by their values, i.e. smallest query index to largest
        qidxs.sort(key=lambda i: queries[i])

        interval_idx = 0 # next interval index to push
        interval_size_heap = [] # (size, end) for all intervals containing next query, min heap by size
        
        for idx in qidxs:
            query = queries[idx]

            # Pop the top element of the heap if it's interval end point is less than the current query
            while interval_size_heap and interval_size_heap[0][1] < query:
                heappop(interval_size_heap)
            
            # Iterate through all intervals who's start value is less than or equal to current query
            while interval_idx < len(intervals) and intervals[interval_idx][0] <= query:
                # If current intervals end value is greater than or equal to current query then it is a valid interval to contain current query
                if intervals[interval_idx][1] >= query:
                    # skip intervals that start later, but have already ended
                    # (we'd pop them to maintain RI anyway)
                    interval_s, interval_e = intervals[interval_idx]
                    interval_size = interval_e-interval_s+1
                    heappush(interval_size_heap, (interval_size, interval_e))
                
                interval_idx += 1

            if interval_size_heap:
                ans[idx] = interval_size_heap[0][0]

        return ans